# Lab2: Optimization I (Solution)

University of California Berkeley

ME C231A, EE C220B, Experiential Advanced Control I

***

These notes were developed by Roya Firoozi and Francesco Borrelli at UC Berkeley. They are protected by U.S. copyright law and by University policy (https://copyright.universityofcalifornia.edu/resources/ownership-course-materials.html).

If you are enrolled in ME C231A/EE C220B you may take notes and make copies of course materials for your own use. You may also share those materials with another student who is registered and enrolled in this course, and with DSP.

You may not reproduce, distribute or display (post/upload) (Links to an external site.) lecture notes or recordings or course materials in any other way — whether or not a fee is charged — without my express written consent. You also may not allow others to do so. If you do so, you may be subject to student conduct proceedings under the Links to an external site.Berkeley Code of Student Conduct, including Sections 102.23 and 102.25.

***

*This* lab includes
1. an introduction to all the optimization problems, modeling toolboxes and solvers that are used in the course,
2. also, you will become familiar with solving linear program and quadratic program using $\texttt{cvxopt}$.

**Optimization Problems:**

Linear programming

Quadratic Programming

Convex Programming

nonlinear/nonconvex

Mixed-integer programming

**Modeling APIs:**

Pyomo http://www.pyomo.org/

CVXPY https://www.cvxpy.org/

PICOS https://picos-api.gitlab.io/picos/index.html

Scipy.optimize.linprog https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

Scipy.optimize.minimize https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

**Solvers:**

IPOPT https://coin-or.github.io/Ipopt/

CLP https://github.com/coin-or/Clp

CBC https://projects.coin-or.org/Cbc

GLPK https://www.gnu.org/software/glpk/

CVXOPT http://cvxopt.org/index.html

MOSEK https://www.mosek.com/

OSQP https://osqp.org/docs/get_started/

**In the optimization module, you will be exposed to some of the above Sovlers/ Modeling Api. In the MPC module we will mostly use Pyomo+IPOPT.**

***

# Linear programming example

Let $x,y,z\in\mathbb{R}$. Use $\texttt{cvxopt}$ to solve the following problem.
\begin{align}
\min_{x,y,z}\ & x+y+z\\
\text{subject to} & -2\le x\\
&-1\le y\\
&-3\le z\\
&x-y+z\ge 4
\end{align}

In [ ]:
import numpy as np
import cvxopt
from cvxopt import matrix, solvers

c = np.ones((3,))   # 1D array
A = np.array([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [-1, 1, -1]])   # 2D array
b = np.array([2,1,3,-4])   # 1D array
c = cvxopt.matrix(c, tc='d')
A = cvxopt.matrix(A, tc='d')
b = cvxopt.matrix(b, tc='d')

sol = cvxopt.solvers.lp(c,A,b)
xOpt = sol['x']
J = sol['primal objective']
print('xOpt =', xOpt)
print('Cost =',J)


     pcost       dcost       gap    pres   dres   k/t
 0: -4.0000e+00 -4.0000e+00  9e+00  1e+00  3e-16  1e+00
 1:  1.7327e+00  1.9400e+00  3e+00  4e-01  5e-16  6e-01
 2:  1.9923e+00  1.9947e+00  4e-02  5e-03  3e-16  7e-03
 3:  1.9999e+00  1.9999e+00  4e-04  5e-05  2e-16  7e-05
 4:  2.0000e+00  2.0000e+00  4e-06  5e-07  3e-16  7e-07
 5:  2.0000e+00  2.0000e+00  4e-08  5e-09  5e-16  7e-09
Optimal solution found.
xOpt = [ 2.00e+00]
[-1.00e+00]
[ 1.00e+00]

Cost = 1.9999999923035239


***

# Quadratic programs and constrained least-squares exercise:

Constrained least-squares problems are useful when your decision parameters are known to reside in a constrained set. Here, we show how to solve a constrained least-squares as a quadratic program. The constrained least-squares problem is of the form
\begin{align}
\min_{x}&\:\|Ax-b\|_2^2\\
\text{s.t. }&l_i\le x_i\le u_i.
\end{align}

Observe that

\begin{align}
\|Ax-b\|_2^2=(x^TA^T-b^T)(Ax-b)=x^TA^TAx-2b^TAx+b^Tb
\end{align}

We can drop the constant term $b^Tb$ in the optimization program and add it back later. The constrained least-squares problem becomes
\begin{align}
\min_{x}&\:\frac{1}{2}x^T2A^TAx-2b^TAx\\
\text{s.t. }&l_i\le x_i\le u_i,
\end{align}
which is a quadratic program.

The commands below solves the constrained least-squares problem using the following $A$ and $b$ for $X \in \mathbb{R}^5$, with $l_i=-0.5$ and $u_i=0.5$.


In [ ]:
n = 5 # dimenstion of x
A = np.random.randn(n, n)
b = np.random.randn(n)
l_i = -0.5
u_i = 0.5


P = 2*A.T@A
q = -2*b.T@A
G = np.concatenate([np.eye(n), -np.eye(n)], axis=0)
h = np.concatenate([u_i*np.ones((n,)), -l_i*np.ones((n,))], axis=0)

P = cvxopt.matrix(P, tc='d')
q = cvxopt.matrix(q, tc='d')
G = cvxopt.matrix(G, tc='d')
h = cvxopt.matrix(h, tc='d')
sol = cvxopt.solvers.qp(P,q,G,h)

print('x*=', sol['x'])
print('p*=', sol['primal objective'])

     pcost       dcost       gap    pres   dres
 0: -5.9961e+00 -1.2635e+01  3e+01  2e+00  2e-16
 1: -5.4857e+00 -9.0213e+00  5e+00  3e-01  2e-16
 2: -5.1904e+00 -5.7734e+00  6e-01  9e-03  1e-15
 3: -5.4115e+00 -5.4778e+00  7e-02  8e-04  1e-16
 4: -5.4222e+00 -5.4256e+00  3e-03  2e-16  2e-16
 5: -5.4228e+00 -5.4230e+00  2e-04  2e-16  6e-16
 6: -5.4228e+00 -5.4228e+00  4e-06  1e-16  3e-16
Optimal solution found.
x*= [ 4.89e-01]
[ 3.67e-01]
[-5.00e-01]
[ 3.74e-01]
[ 5.00e-01]

p*= -5.422798668077884


# CVXPY Exercise

In [ ]:
# Now we define and solve the same quadratic problem using CVXPY

import cvxpy as cv

# n = 5 # dimenstion of x
# A = np.random.randn(n, n)
# b = np.random.randn(n)
l_i = -0.5
u_i = 0.5

P = 2*A.T@A
q = -2*A.T@b
G = np.concatenate([np.eye(n), -np.eye(n)], axis=0)
h = np.concatenate([u_i*np.ones((n,)), -l_i*np.ones((n,))], axis=0)

x = cv.Variable(n)
prob = cv.Problem(cv.Minimize((1/2)*cv.quad_form(x, P) + q.T @ x),
                 [G @ x <= h])
print(prob.solve())

-5.4227987179797825


***

# Linear Programming Exercise

For the following problems, write a linear program and solve using $\texttt{cvxopt}$. Determine the traits of the problem (i.e., feasible or infeasible, bounded or unbounded, has a unique optimum?)

\begin{align}
\min_{z_1,z_2}~ &  3z_1 + 2z_2 \\
& z_1 \geq 0  \\
& z_2 \geq 0
\end{align}

In [ ]:
import cvxopt
from cvxopt import matrix, solvers


c = np.array([3,2])   # 1D array
A = -np.eye(2)   # 2D array
b = np.zeros((2,1))   # 1D array
c = cvxopt.matrix(c, tc='d')
A = cvxopt.matrix(A, tc='d')
b = cvxopt.matrix(b, tc='d')

sol = cvxopt.solvers.lp(c,A,b)
xOpt = sol['x']
J = sol['primal objective']
print('xOpt =', xOpt)
print('Cost =',J)


Optimal solution found.
xOpt = [ 0.00e+00]
[ 0.00e+00]

Cost = 0.0


***

\begin{align}
\min_{z_1,z_2}~ &  z_1 \\
& z_1 \geq 0  \\
& z_2 \geq 0
\end{align}

In [ ]:
c = np.array([1,0])   # 1D array
A = -np.eye(2)   # 2D array
b = np.zeros((2,1))   # 1D array
c = cvxopt.matrix(c, tc='d')
A = cvxopt.matrix(A, tc='d')
b = cvxopt.matrix(b, tc='d')

sol = cvxopt.solvers.lp(c,A,b)
xOpt = sol['x']
J = sol['primal objective']
print('xOpt =', xOpt)
print('Cost =',J)


Optimal solution found.
xOpt = [ 0.00e+00]
[ 0.00e+00]

Cost = 0.0


***

\begin{align}
\min_{z_1,z_2}~ &  -5z_1 -7z_2 \\
\text{s.t. } & -3z_1 +2z_2 \leq 30  \\
& -2z_1 + z_2 \leq 12  \\
& z_1 \geq 0  \\
& z_2 \geq 0  
\end{align}

In [ ]:
c = np.array([-5, -7])   # 1D array
A = -np.eye(2)   # 2D array
b = np.zeros((2,1))   # 1D array
c = cvxopt.matrix(c, tc='d')
A = cvxopt.matrix(A, tc='d')
b = cvxopt.matrix(b, tc='d')

sol = cvxopt.solvers.lp(c,A,b)
xOpt = sol['x']
J = sol['primal objective']
print('xOpt =', xOpt)
print('Cost =',J)

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -0.0000e+00  4e+00  1e+00  1e+00  1e+00
 1: -1.1827e+02 -0.0000e+00  9e+03  3e+01  2e+01  1e+02
 2: -5.7251e+01 -0.0000e+00  2e+01  1e+00  1e+00  6e+01
 3: -5.9204e+03 -0.0000e+00  6e+03  2e+00  2e+00  6e+03
 4: -5.9280e+05 -0.0000e+00  6e+05  2e+00  2e+00  6e+05
 5: -5.9281e+07 -0.0000e+00  6e+07  2e+00  2e+00  6e+07
Certificate of dual infeasibility found.
xOpt = [ 1.57e-01]
[ 3.09e-02]

Cost = -1.0


***

\begin{align}
\min_{z_1,z_2}~ &  z_1 -z_2 \\
\text{s.t. } & z_1 -z_2 \geq 2  \\
& 2z_1 + z_2 \geq 1 \\
& z_1 \geq 0  \\
& z_2 \geq 0  
\end{align}

In [ ]:
c = np.array([1, -1])   # 1D array
A = np.array([[-1, 1],
              [-2, -1],
              [-1, 0],
              [0, -1]])   # 2D array
b = np.array([-2, -1, 0, 0])  # 1D array
c = cvxopt.matrix(c, tc='d')
A = cvxopt.matrix(A, tc='d')
b = cvxopt.matrix(b, tc='d')

sol = cvxopt.solvers.lp(c,A,b)
xOpt = sol['x']
J = sol['primal objective']
print('xOpt =', xOpt)
print('Cost =',J)

     pcost       dcost       gap    pres   dres   k/t
 0:  1.0000e+00  5.0000e+00  1e+01  2e+00  4e+00  1e+00
 1:  2.2797e+00  2.9813e+00  2e+00  2e-01  4e-01  4e-01
 2:  2.0024e+00  2.0091e+00  1e-02  2e-03  5e-03  3e-03
 3:  2.0000e+00  2.0001e+00  1e-04  2e-05  5e-05  3e-05
 4:  2.0000e+00  2.0000e+00  1e-06  2e-07  5e-07  3e-07
 5:  2.0000e+00  2.0000e+00  1e-08  2e-09  5e-09  3e-09
Optimal solution found.
xOpt = [ 3.11e+00]
[ 1.11e+00]

Cost = 2.000000002422302


***

\begin{align}
\min_{z_1,z_2}~ & 3z_1 + z_2  \\
\text{s.t. } & z_1 - z_2 \leq 1  \\
& 3z_1 + 2z_2 \leq 12  \\
& 2z_1 + 3z_2 \leq 3  \\
& -2z_1 +3z_2 \geq 9  \\
& z_1 \geq 0  \\
& z_2 \geq 0
\end{align}

In [ ]:
c = np.array([3, 1])   # 1D array
A = np.array([[1, -1],
              [3, 2],
              [2, 3],
              [2, -3],
              [-1, 0],
              [0, -1]])   # 2D array
b = np.array([1, 12, 3, -9, 0, 0])  # 1D array
c = cvxopt.matrix(c, tc='d')
A = cvxopt.matrix(A, tc='d')
b = cvxopt.matrix(b, tc='d')

sol = cvxopt.solvers.lp(c,A,b)
xOpt = sol['x']
J = sol['primal objective']
print('xOpt =', xOpt)
print('Cost =',J)

     pcost       dcost       gap    pres   dres   k/t
 0:  4.4339e+00 -5.9606e+00  5e+01  1e+00  3e+00  1e+00
 1: -4.2130e+00 -7.6802e+00  4e+01  4e-01  1e+00  1e+00
 2: -8.7331e-01 -9.1562e-01  4e+01  2e-01  7e-01  2e+00
 3: -4.4861e+00  2.2130e+02  1e+04  6e-01  2e+00  2e+02
 4: -4.4731e+00  2.2991e+04  1e+06  6e-01  2e+00  2e+04
 5: -4.4731e+00  2.3000e+06  1e+08  6e-01  2e+00  2e+06
 6: -4.4731e+00  2.3000e+08  1e+10  6e-01  2e+00  2e+08
Certificate of primal infeasibility found.
xOpt = None
Cost = None


***

# Quadratic Programming Exercise
For the following problems write a quadratic program and solve using $\texttt{cvxopt}$. For each constraint determine if it is active or inactive.
\begin{align}
\min_{z_1,z_2}~ &  z_1^2 + z_2^2 \\
& z_1 \geq 1  \\
& z_2 \geq 1
\end{align}

In [ ]:
import cvxopt

P = 2*np.eye(2)
q = np.zeros((2,1))
G = -np.eye(2)
h = -np.ones((2,1))

P = cvxopt.matrix(P, tc='d')
q = cvxopt.matrix(q, tc='d')
G = cvxopt.matrix(G, tc='d')
h = cvxopt.matrix(h, tc='d')
sol = cvxopt.solvers.qp(P,q,G,h)

print('x*=', sol['x'])
print('p*=', sol['primal objective'])

     pcost       dcost       gap    pres   dres
 0:  2.2222e-01  1.1111e+00  1e+00  2e+00  2e-16
 1:  7.2114e-01  1.6808e+00  2e-02  4e-01  3e-16
 2:  2.0165e+00  2.0000e+00  2e-02  2e-16  3e-15
 3:  2.0002e+00  2.0000e+00  2e-04  0e+00  6e-16
 4:  2.0000e+00  2.0000e+00  2e-06  0e+00  0e+00
Optimal solution found.
x*= [ 1.00e+00]
[ 1.00e+00]

p*= 2.000001642255182


***

\begin{align}
\min_{z_1,z_2}~ &  2z_1^2 + 7z_2^2 \\
& z_1 \geq -3  \\
& 2 \geq z_2
\end{align}

In [ ]:
P = 2*np.diag([2, 7])
q = np.zeros((2,1))
G = np.diag([-1, 1])
h = np.array([3, 2])

P = cvxopt.matrix(P, tc='d')
q = cvxopt.matrix(q, tc='d')
G = cvxopt.matrix(G, tc='d')
h = cvxopt.matrix(h, tc='d')
sol = cvxopt.solvers.qp(P,q,G,h)

print('x*=', sol['x'])
print('p*=', sol['primal objective'])

     pcost       dcost       gap    pres   dres
 0:  8.4444e-01 -4.4178e+00  5e+00  3e-17  5e+00
 1:  1.0131e-05 -1.3367e-01  1e-01  8e-17  4e-02
 2:  1.0134e-09 -1.3368e-03  1e-03  1e-16  4e-04
 3:  1.0134e-13 -1.3368e-05  1e-05  2e-16  4e-06
 4:  1.0134e-17 -1.3368e-07  1e-07  7e-17  4e-08
 5:  1.0134e-21 -1.3368e-09  1e-09  1e-16  4e-10
Optimal solution found.
x*= [ 8.58e-12]
[ 1.11e-11]

p*= 1.0133883260344452e-21


***

\begin{align}
\min_{z_1,z_2}~ &  z_1^2 + z_2^2 \\
& z_1 \leq -3  \\
& z_2 \leq 4 \\
& 0 \geq 4z_1 + 3z_2
\end{align}

In [ ]:
P = 2*np.eye(2)
q = np.zeros((2,1))
G = np.array([[1, 0],
              [0, 1],
              [4, 3]])
h = np.array([-3, 4, 0])

P = cvxopt.matrix(P, tc='d')
q = cvxopt.matrix(q, tc='d')
G = cvxopt.matrix(G, tc='d')
h = cvxopt.matrix(h, tc='d')
sol = cvxopt.solvers.qp(P,q,G,h)

print('x*=', sol['x'])
print('p*=', sol['primal objective'])

     pcost       dcost       gap    pres   dres
 0:  2.7778e+00  1.1444e+01  2e+01  1e+00  2e+01
 1:  9.5297e+00  1.4013e+01  6e+00  3e-01  7e+00
 2:  9.7717e+00  6.8639e+00  3e+00  7e-16  5e-15
 3:  9.0070e+00  8.9447e+00  6e-02  4e-16  2e-15
 4:  9.0001e+00  8.9994e+00  6e-04  5e-17  4e-15
 5:  9.0000e+00  9.0000e+00  6e-06  4e-16  9e-16
Optimal solution found.
x*= [-3.00e+00]
[-6.92e-07]

p*= 9.000000692261725


***

\begin{align}
\min~ &  \frac{1}{2}(z_1^2 + z_2^2 +0.1z_3^2) + 0.55z_3 \\
& z_1 + z_2 + z_3 = 1  \\
& z_1 \geq 0 \\
& z_2 \geq 0 \\
& z_3 \geq 0
\end{align}

In [ ]:
P = np.diag([1, 1, 0.1])
q = np.array([0, 0, 0.55])
G = -np.eye(3)
h = np.zeros((3,1))
A = np.ones((1,3)) # special case
b = np.array([1])

P = cvxopt.matrix(P, tc='d')
q = cvxopt.matrix(q, tc='d')
G = cvxopt.matrix(G, tc='d')
h = cvxopt.matrix(h, tc='d')
A = cvxopt.matrix(A, tc='d')
b = cvxopt.matrix(b, tc='d')
sol = cvxopt.solvers.qp(P,q,G,h,A,b)

print('x*=', sol['x'])
print('p*=', sol['primal objective'])

NameError: ignored